#  Raw to bronze notebook

En esta notebook denominada RawToBronze simplemente copiamos la información de los datos de acciones mediante la API yfinance, y se almacena en la capa Bronze del data lake. En esta etapa, se configura el ticker de la acción de Apple para obtener los datos financieros diarios, que incluyen información como la apertura, cierre, máximo, mínimo, volumen de transacciones, y la fecha. Esta notebook debe ser ejecutada en Azure Synapse.

## Parámetros
- **Asset (str)**: Indica el activo financiero que se desea analizar. Es el nombre del directorio en el que se guardará. 'apple' es el valor por defecto.
- **Ticker (str)**: Indica el nombre del activo financiero en la API de yfinance. 'AAPL' es el activo por defecto.
- **start_date (str)**: Fecha de inicio del rango de datos en formato 'YYYY-MM-DD'. Por defecto, es "2023-01-01".
- **end_date (str)**: Fecha de fin del rango de datos en formato 'YYYY-MM-DD'.Si no se proporciona, se utiliza la fecha del día anterior al actual.

In [ ]:
# Parameters
asset = "apple"   
ticker = "AAPL"
start_date = "2023-01-01"
end_date = None

In [ ]:
# En primer lugar, deberemos instalar yfinance si no lo tenemos. Esta es la API de Yahoo Finance, desde la cual podremos obtener la información que deseamos sobre los activos financieros.
!pip install yfinance

In [ ]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from datetime import datetime, timedelta
from delta.tables import DeltaTable 
import pandas as pd
import os
import yfinance as yf

In [ ]:

def descargar_datos(ticker, start_date="2023-01-01", end_date=None):
    """
    Descarga datos financieros de un ticker utilizando la librería yfinance.

    Parámetros:
    ----------
    ticker : str
        Símbolo del ticker para el cual se desea descargar los datos (por ejemplo, "AAPL" para Apple).
    
    start_date : str, opcional
        Fecha de inicio del rango de datos en formato 'YYYY-MM-DD'. 
        Por defecto, es "2023-01-01".
    
    end_date : str, opcional
        Fecha de fin del rango de datos en formato 'YYYY-MM-DD'. 
        Si no se proporciona, se utiliza la fecha del día anterior al actual.
    
    Retorna:
    -------
    pandas.DataFrame
        Un DataFrame con los datos financieros del ticker para el rango de fechas especificado.

    Ejemplo:
    --------
    >>> datos = descargar_datos("AAPL")
    >>> print(datos)
    """
    if end_date is None:
        end_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    
    data_input = yf.download(ticker, start=start_date, end=end_date)
    return data_input


In [ ]:
container= "datalake"
storage_account_name = 'nticmasterstg'
data_lake_container = f'abfss://{container}@{storage_account_name}.dfs.core.windows.net'
landing_folder = 'bronze'  

landing_table_path = f"{data_lake_container}/{landing_folder}/{asset}"  

In [ ]:
data_input = descargar_datos(ticker, start_date, end_date)

# Convertir el índice a datetime, y eliminar el índice para así evitar la pérdida de esta columna al guardar el dataframe en Spark.
data_input.index = pd.to_datetime(data_input.index)

data_input = data_input.reset_index()

data = spark.createDataFrame(data_input)

#Aplicamos este bucle para eliminar problemas con los nombres de las columnas al generar el Spark dataframe.
nombres_columnas = [col.replace(" ", "_").replace(",", "_").replace(";", "_").replace("{", "_").replace("}", "_")
                    .replace("(", "_").replace(")", "_").replace("\n", "_").replace("\t", "_").replace("=", "_") 
                    for col in data.columns]

data = data.toDF(*nombres_columnas)

In [ ]:
print('Creando una nueva tabla Delta...') 
data.write.format('delta').mode("overwrite").save(landing_table_path)
print('La tabla {asset} se ha guardado en la ruta {landing_table_path}')

delta_df = spark.read.format("delta").load(landing_table_path)
print('Mostrando 5 líneas del contenido de la tabla Delta...')
delta_df.show(5)

conteo_filas = delta_df.count()

print(f'La tabla Delta tiene {conteo_filas} filas en total')